In [13]:
import init

In [14]:
from urllib.parse import urlparse
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
from webspot.request.html_requester import HtmlRequester
from webspot.graph.graph_loader import GraphLoader
from webspot.detect.detectors.pagination import PaginationDetector
from webspot.detect.detectors.plain_list import PlainListDetector
from webspot.detect.utils.transform_html_links import transform_url

In [16]:
# url = 'https://quotes.toscrape.com'
# url = 'https://books.toscrape.com'
url = 'https://cuiqingcai.com/archives/'
# url = 'https://cuiqingcai.com/archives/page/2/'

html_requester = HtmlRequester(url=url)
html_requester.run()

graph_loader = GraphLoader(html=html_requester.html_, json_data=html_requester.json_data)
graph_loader.run()

pagination_detector = PaginationDetector(html_requester=html_requester, graph_loader=graph_loader)
plain_list_detector = PlainListDetector(html_requester=html_requester, graph_loader=graph_loader)

/Users/marvzhang/projects/crawlab-team/webspot/venv/lib/python3.10/site-packages/dgl/heterograph.py:72: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning('Recommend creating graphs by `dgl.graph(data)`'
/Users/marvzhang/projects/crawlab-team/webspot/venv/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [17]:
feature_tag_a_idx = np.argwhere(np.array(graph_loader.nodes_features_enc.feature_names_) == 'tag=a')[0]
link_nodes_idx = np.argwhere(graph_loader.nodes_features_tensor[:, feature_tag_a_idx].T[0] == 1)[0].detach().numpy()
link_nodes_idx

array([ 12,  25,  27,  29,  31,  33,  35,  37,  67,  74,  81,  88,  95,
       102, 109, 116, 123, 130, 137, 144, 151, 158, 165, 172, 179, 186,
       193, 200, 207, 214, 221, 228, 235, 242, 249, 256, 263, 270, 277,
       284, 291, 298, 305, 312, 319, 326, 333, 340, 344, 346, 347, 368,
       372, 376, 393, 400, 403, 406, 409, 412, 415, 418, 421, 424, 427,
       430, 433, 436, 439, 442, 445, 448, 451, 454, 457, 460, 463, 466,
       469, 472, 475, 478, 487, 492, 497, 502, 507, 512, 517, 522, 527,
       532, 537, 542, 547, 552, 557, 562, 567, 572, 577, 582, 587, 592,
       597, 602, 607, 612, 617, 621, 622, 623, 624, 625, 626, 627, 628,
       629, 630, 631, 632, 633, 634, 635, 636, 637, 638, 639, 640, 641,
       642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654,
       655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 665, 666, 667,
       668, 669, 670, 671, 672, 673, 674, 675, 676, 677, 678, 679, 680,
       681, 682, 683, 684, 685, 686, 687, 688, 689, 690, 691, 69

In [18]:
nodes_a = graph_loader.nodes[link_nodes_idx]

nodes_a[0]

{'id': 57,
 'parent_id': 56,
 'features': [('tag', 'a'),
  ('href', '/'),
  ('class', 'brand'),
  ('rel', 'start')],
 'text': None}

In [19]:
hrefs = [
    transform_url(html_requester.url, n.attrs.get('href'))
    for n in pagination_detector.link_nodes
    if n.attrs.get('href') is not None 
    and urlparse(html_requester.url).netloc in transform_url(html_requester.url, n.attrs.get('href'))
]

hrefs

['https://cuiqingcai.com/archives/',
 'https://cuiqingcai.com/tags/',
 'https://cuiqingcai.com/categories/',
 'https://cuiqingcai.com/about/',
 'https://cuiqingcai.com/message/',
 'https://cuiqingcai.com/36065.html',
 'https://cuiqingcai.com/36064.html',
 'https://cuiqingcai.com/36063.html',
 'https://cuiqingcai.com/36062.html',
 'https://cuiqingcai.com/36061.html',
 'https://cuiqingcai.com/36060.html',
 'https://cuiqingcai.com/36059.html',
 'https://cuiqingcai.com/36058.html',
 'https://cuiqingcai.com/36057.html',
 'https://cuiqingcai.com/36056.html',
 'https://cuiqingcai.com/17777.html',
 'https://cuiqingcai.com/2022114.html',
 'https://cuiqingcai.com/2022113.html',
 'https://cuiqingcai.com/2022111.html',
 'https://cuiqingcai.com/2022112.html',
 'https://cuiqingcai.com/2022104.html',
 'https://cuiqingcai.com/2022103.html',
 'https://cuiqingcai.com/2022102.html',
 'https://cuiqingcai.com/202293.html',
 'https://cuiqingcai.com/2022101.html',
 'https://cuiqingcai.com/36054.html',
 'http

In [20]:
hrefs_frag = [[f for f in urlparse(href).path.split('/') if f != ''] for href in hrefs if href.startswith('http')]
hrefs_frag

[['archives'],
 ['tags'],
 ['categories'],
 ['about'],
 ['message'],
 ['36065.html'],
 ['36064.html'],
 ['36063.html'],
 ['36062.html'],
 ['36061.html'],
 ['36060.html'],
 ['36059.html'],
 ['36058.html'],
 ['36057.html'],
 ['36056.html'],
 ['17777.html'],
 ['2022114.html'],
 ['2022113.html'],
 ['2022111.html'],
 ['2022112.html'],
 ['2022104.html'],
 ['2022103.html'],
 ['2022102.html'],
 ['202293.html'],
 ['2022101.html'],
 ['36054.html'],
 ['202292.html'],
 ['202291.html'],
 ['36055.html'],
 ['36052.html'],
 ['36053.html'],
 ['202282.html'],
 ['202272.html'],
 ['202281.html'],
 ['202261.html'],
 ['202246.html'],
 ['202271.html'],
 ['202262.html'],
 ['202253.html'],
 ['202251.html'],
 ['202242.html'],
 ['202232.html'],
 ['202252.html'],
 ['202241.html'],
 ['202231.html'],
 ['archives', 'page', '2'],
 ['archives', 'page', '17'],
 ['archives', 'page', '2'],
 ['archives'],
 ['categories'],
 ['tags'],
 ['categories', 'C-C'],
 ['categories', 'HTML'],
 ['categories', 'Java'],
 ['categories', 

In [21]:
html_requester.url

'https://cuiqingcai.com/archives/'

In [22]:
current_fragments = pagination_detector.current_url_path_fragments
current_fragments

['archives']

In [23]:
pagination_detector.run()

In [54]:
nodes_texts = np.array([len(set(n.text.lower().split(' ')).intersection(pagination_detector.keywords)) > 0 if n.text else False for n in pagination_detector.internal_link_nodes])
nodes_texts * 1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [58]:
scores_idx = np.argwhere(pagination_detector.scores >= 1.).T[0]
scores_idx

array([ 0, 45, 46, 47, 48])

In [63]:
np.argmax(pagination_detector.scores[scores_idx])

3